In [25]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'matplotlib'

# Montando DataFrame

In [2]:
dias = [
    '2022-01-31',
    '2022-02-01', 
    '2022-02-02', 
    '2022-02-03', 
    '2022-02-04',
    '2022-02-05',
    '2022-02-06',
]

df = pd.DataFrame()

for dia in dias:
    with open(f'datalake/extract_date={dia}.json') as file:
        text_id = []
        user_id = []
        user_name = []
        number_of_followers = []
        retweeted_text_id = []
        retweeted_text = []
        retweeted_user_id = []
        retweeted_user_name = []
        retweeted_user_number_of_followers = []
        retweet_count = []
        search_tweets = json.load(file)
        for tweet in search_tweets:
            text_id.append(tweet['id'])
            user_id.append(tweet['user']['id'])
            user_name.append(tweet['user']['screen_name'])
            number_of_followers.append(tweet['user']['followers_count'])
            retweeted_text_id.append(tweet['retweeted_status']['id'])
            retweeted_text.append(tweet['retweeted_status']['text'])
            retweeted_user_id.append(tweet['retweeted_status']['user']['id'])
            retweeted_user_name.append(tweet['retweeted_status']['user']['screen_name'])
            retweeted_user_number_of_followers.append(tweet['retweeted_status']['user']['followers_count'])
            retweet_count.append(tweet['retweeted_status']['retweet_count'])
        _ = pd.DataFrame(list(zip(
            text_id, user_id, user_name, number_of_followers,
            retweeted_text_id, retweeted_text, retweeted_user_id,
            retweeted_user_name, retweeted_user_number_of_followers,
            retweet_count
        )))        
    df = pd.concat([df, _], axis=0)

map = {
    0: 'text_id', 1: 'user_id', 2: 'user_name', 3: 'number_of_followers',
    4: 'retweeted_text_id', 5: 'retweeted_text', 6: 'retweeted_user_id',
    7: 'retweeted_user_name', 8: 'retweeted_user_number_of_followers',
    9: 'retweet_count'
}
df = df.rename(columns=map)

# Pré-análise

In [3]:
print(f'Quantidade de linhas: {df.shape[0]}')
print(f'Quantidade de colunas: {df.shape[1]}')

Quantidade de linhas: 3168
Quantidade de colunas: 10


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3168 entries, 0 to 433
Data columns (total 10 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   text_id                             3168 non-null   int64 
 1   user_id                             3168 non-null   int64 
 2   user_name                           3168 non-null   object
 3   number_of_followers                 3168 non-null   int64 
 4   retweeted_text_id                   3168 non-null   int64 
 5   retweeted_text                      3168 non-null   object
 6   retweeted_user_id                   3168 non-null   int64 
 7   retweeted_user_name                 3168 non-null   object
 8   retweeted_user_number_of_followers  3168 non-null   int64 
 9   retweet_count                       3168 non-null   int64 
dtypes: int64(7), object(3)
memory usage: 272.2+ KB


In [5]:
df['text_id'] = df['text_id'].astype('str')
df['user_id'] = df['user_id'].astype('str')
df['user_name'] = df['user_name'].astype('str')
df['retweeted_text_id'] = df['retweeted_text_id'].astype('str')
df['retweeted_text'] = df['retweeted_text'].astype('str')
df['retweeted_user_id'] = df['retweeted_user_id'].astype('str')
df['retweeted_user_name'] = df['retweeted_user_name'].astype('str')

In [6]:
df.describe(include='O')

,text_id,user_id,user_name,retweeted_text_id,retweeted_text,retweeted_user_id,retweeted_user_name
count,3168,3168,3168,3168,3168,3168,3168
unique,900,869,869,218,218,181,181
top,1487735300818407425,1284133047344955393,FabimBrasil,1487558809220435971,Militares pedem explicações do TSE sobre trans...,1090291692744466433,SigaGazetaBR
freq,7,21,21,778,778,1013,1013


# Análise Exploratória de Dados

In [24]:
# Top 10 usuários mais retweetados
_ = df.copy()
_['count'] = 1
_ = _.groupby(['retweeted_user_id', 'retweeted_user_name']).sum()
_[['count']].sort_values('count', ascending=False).head(10)


,,count
retweeted_user_id,retweeted_user_name,
1090291692744466433,SigaGazetaBR,1013
1159467623110848512,brom_elisa,434
1300199162684768257,ibopeenquetes,280
1478442160764731394,JooBati47318744,70
3331429624,Metropoles,70
1267242805132431361,DestroAvante,70
2902359105,freu_rodrigues,70
787361839827062788,forumpandlr,56
14594813,folha,49


In [10]:
# Top 10 tweets mais retweetados
_ = df[['retweeted_user_name', 'retweeted_text_id', 'retweeted_text', 'retweet_count']].drop_duplicates()
_.sort_values('retweet_count', ascending=False)[:10]

,retweeted_user_name,retweeted_text_id,retweeted_text,retweet_count
8,jairbolsonaro,1469951475728199682,- Vale lembrar. O vencedor das eleições de 202...,8030
0,forumpandlr,1488975260049358856,O deputado Alexandre Frota (PSDB-SP) apresento...,3209
429,LulaOficial,1482378218350993411,Eu disse nesse começo de ano ao jornal britâni...,2305
62,SigaGazetaBR,1487558809220435971,Militares pedem explicações do TSE sobre trans...,1063
88,brom_elisa,1487467272348311554,"Há mais de um mês,as Forças Armadas apresentar...",1055
68,Juba714,1479600797075783680,"2022 um ano de eleições,um ano de guerra,se pr...",942
0,folha,1487915286619643912,Socialistas vencem eleições em Portugal e deve...,868
0,SigaGazetaBR,1489361766261870596,Bolsonaro avalia Daniel Silveira como melhor o...,604
11,dep_padrejoao,1487801068159479812,Eleições 2022. #Pesquisa #DataPovo #Eleições2022,539
20,eixopolitico,1488973191078940676,🇧🇷 VACINAR PARA VOTAR?\n\nDeputado Alexandre F...,477
